# Determine the molecular gas mass of a z=1 star-forming galaxy

We, in the present notebook, use known star formation rate from a galaxy and use it to estimate the molecular gas mass in the galaxy.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as con
import astropy.units as u
from astropy.cosmology import Planck15 as cosmo

### Deriving Luminosity from star formation rate

The formula to be used is from Kennicutt (1998) and stated below:

$$SFR (in \ M_\odot year^{-1}) = 4.5 \times 10^{-44} \cdot L_{FIR} (in \ erg \ s^{-1})$$

$$\Rightarrow L_{FIR} (in \ erg \ s^{-1}) = \frac{SFR (in \ M_\odot year^{-1}) }{4.5 \times 10^{-44}}$$

This formula is used to compute the lumminosity density from the star formation rate.

In [2]:
sfr = 30
kappa = 4.5e-44

lfir = sfr/kappa ## in erg/sec
print('The infrared luminosity (in cgs units) is: {:.2e}'.format(lfir))

The infrared luminosity (in cgs units) is: 6.67e+44


Now we want to compute the CO luminosity from this IR luminosity. We can use the following equation from Carilli & Walter (2013),

$$\log L_{IR} = 1.13 \cdot \log L'_{CO} + 0.53 $$

$$\Rightarrow \log L'_{CO} = \frac{\log L_{IR} - 0.53}{1.13}$$

Here, $\log L_{IR}$ in in units of $L_\odot$ and $\log L'_{CO}$ would be in $K \ km \ s^{-1} \ pc^2$.

In [3]:
lsun = ((con.L_sun).value)*(1e7)
lfir_sun = lfir/lsun
logco = (np.log10(lfir_sun)-0.53)/1.13
lco_diff_units = 10**(logco)

print('The CO luminosity is (in log K km s-1 pc2): {:.2e}'.format(lco_diff_units))

The CO luminosity is (in log K km s-1 pc2): 3.01e+09


### Choose appropriate frequency to observe with ALMA

We now want to choose the appropriate frequency for observation. To do this we want to first compute the observed frequency of various CO lines at redshift $z \sim 1$.

In [4]:
lines = np.array(['CO10', 'CO21', 'CO32', 'C043', 'CO54', 'CO65', 'CO75'])

rest_frame = np.array([115.271204, 230.537990, 345.795989, 461.040770, 576.267904, 691.473090, 806.651806])

redshift = 1.036

obs_freq = rest_frame/(1+redshift)

for i in range(len(lines)):
    print(lines[i] + '\t' + str(obs_freq[i]))

CO10	56.616504911591356
CO21	113.23083988212181
CO32	169.8408590373281
C043	226.44438605108056
CO54	283.0392455795678
CO65	339.6233251473477
CO75	396.1944037328094


### CO (3-2) luminosity

It is evident from the above observed frequencies that frequency corresponding to CO(3-2) is best for ALMA observations. Hence we want to use CO(3-2). And we want to convert our $L'_{CO}$ into $L'_{3-2}$, by multiplying a factor of 0.6 (from Carilli & Walter 2013).

In [5]:
lco_32 = lco_diff_units*0.6

Adding the lensing effect,

In [6]:
lco_32_lens = 4.3*lco_32

### Calculating flux density

Now we want to compute the flux density from the calculated CO luminosity using a formula from Solomon et al. (1997):

$$ L'_{CO} = 3.25 \times 10^7 S_{CO}\Delta V \nu_{obs}^{-2} D_L^2 (1+z)^{-3}$$

with $S_{CO}\Delta V$ in $km \ s^{-1}$, $\nu_{obs}$ in $GHz$ and $D_L$ (which is luminosity distance -- we calculated this from the `Planck18` cosmology from `astropy.cosmology` module) in $Mpc$. Since, we want to measure the flux density (that is, $S_{CO} \Delta V$), we can adjust above equation as,

$$S_{CO} \Delta V = \frac{L'_{CO} \cdot \nu_{obs}^2 (1+z)^3}{3.25 \times 10^7 \cdot D_L^2}$$

Above formula is adapted below,

In [7]:
dl = (cosmo.luminosity_distance(redshift)).value
freq1 = obs_freq[2]

aa = (lco_32_lens*freq1*freq1*((1+redshift)**3))
bb = dl*dl*(3.25e7)

flux_s = aa/bb

print('Flux density (in Jy km-1) is: {:.2e}'.format(flux_s))

Flux density (in Jy km-1) is: 1.16e+00


Since the above computed flux density also include the velosity dispersion, we want to divide it with an appropriate value of the velosity dispersion to get the flux density in the units os $Jy$, which is done below:

In [8]:
flux_density = flux_s/200
print('Flus density (in Jy): {:.2e}'.format(flux_density))

Flus density (in Jy): 5.78e-03


### Mass of H2

We now want to compute the mass of moleculer $H_2$ from the CO luminosity we calculated earlier. We use the conversion factor of $\alpha_{CO} = 4 M_\odot (K km s^{-1} pc^2)^{-1}$ from Carilli & Walter (2013). Note that this conversion factor will give the mass of $H_2$ for CO (1-0) transition luminosity. Hence we want to use the same luminosity for conversion.

In [9]:
mass_h2 = lco_diff_units*4
print('Mass of H2 (in M_sun): {:.2e}'.format(mass_h2))

Mass of H2 (in M_sun): 1.20e+10
